In [51]:
import arrow
import json
import pandas as pd
import numpy as np
import requests

%matplotlib inline 
import matplotlib.pyplot as plt

In [52]:
#endpoint = 'http://localhost:8000'
endpoint = 'http://electricitymap-api.tmrow.co'
country_codes = ['DK', 'ES', 'PT']
bounding_boxes = {
    'DK': [
        [13.30324, 57.751949], # NE
        [7.60472, 54.509132], # SW
    ],
    'ES': [
        [4.32788, 43.789959],
        [-9.560789, 36.63546]
    ],
    'PT': [
        [-6.18931, 42.154121],
        [-9.5, 37]
    ]
}
start, end = [
    arrow.get(2016, 11, 19),
    arrow.get(2016, 12, 1)
]
time_span = map(lambda o: o[0], arrow.Arrow.span_range('hour', start, end))

In [53]:
r = requests.session()

In [54]:
def fetch_production(country_code, t):
    url = '%s/v1/production' % endpoint
    params = {
        'countryCode': country_code,
        'datetime': t.to('utc').isoformat()
    }
    obj = r.get(url, params=params).json()
    if not obj: return
    return obj if (t - arrow.get(obj['datetime'])).total_seconds() < 3600 else None
print time_span[0], fetch_production(country_codes[0], time_span[0])

2016-11-19T00:00:00+00:00

ValueError: No JSON object could be decoded

In [ ]:
# Fetch all country production data
dates = map(lambda t: pd.Timestamp(t.datetime), time_span)
df_wind = pd.DataFrame(columns=country_codes)
df_solar = pd.DataFrame(columns=country_codes)

for country_code in country_codes:
    print 'Fetching %s..' % country_code
    for t in time_span:
        o = fetch_production(country_code, t)
        if not o: continue
        t = arrow.get(o['datetime']).datetime
        if o.get('production', {}).get('wind', None) is not None:
            df_wind.loc[t, country_code] = o['production']['wind']
        if o.get('production', {}).get('solar', None) is not None:
            df_solar.loc[t, country_code] = o['production']['solar']
df_wind = df_wind.sort_index()
df_solar = df_solar.sort_index()
# TODO: write in a cache

In [ ]:
df_wind.plot(marker='x', title='Wind')
df_solar.plot(marker='x', title='Solar')

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from feeder.parsers import weather

# Force a reload
reload(weather)

def fetch_best_forecast(t):
    return weather.fetch_weather(t, compress=False, useCache=True)

In [ ]:
t = time_span[0]
obj = fetch_best_forecast(t)

In [ ]:
def get_measurements(country_code, t, obj):
    # Return list of measurements inside the country
    header_before = obj['forecasts'][0]['wind'][0]['header']
    header_after  = obj['forecasts'][1]['wind'][0]['header']
    t_before = arrow.get(header_before['refTime']).replace(hours=+header_before['forecastTime'])
    t_after  = arrow.get( header_after['refTime']).replace(hours=+header_after['forecastTime'])

    grid_origin = [header_before['lo1'], header_before['la1']]
    grid_delta = [header_before['dx'], header_before['dy']]
    grid_n = [header_before['nx'], header_before['ny']]

    speeds = []
    dswrfs = []
    bb = bounding_boxes[country_code]

    for i in range(grid_n[0]):
        for j in range(grid_n[1]):
            lon = grid_origin[0] + i * grid_delta[0]
            if lon > 180: lon -= 360
            lat = grid_origin[1] - j * grid_delta[1]
            p = j * grid_n[0] + i
            if lon <= bb[0][0] and lat <= bb[0][1] and \
                lon >= bb[1][0] and lat >= bb[1][1]:
                # We're in the box!
                x = (t.timestamp - t_before.timestamp)/(t_after.timestamp - t_before.timestamp)
                if abs(x) > 1: raise Exception('t is outside of bounds!')
                u = obj['forecasts'][0]['wind'][0]['data'][p] * (1-x) + obj['forecasts'][1]['wind'][0]['data'][p] * x
                v = obj['forecasts'][0]['wind'][1]['data'][p] * (1-x) + obj['forecasts'][1]['wind'][1]['data'][p] * x
                s = obj['forecasts'][0]['solar']['data'][p] * (1-x)   + obj['forecasts'][1]['solar']['data'][p] * x

                speeds.append(np.sqrt(u*u + v*v))
                dswrfs.append(s)
    
    return (speeds, dswrfs)

t = arrow.get(2016, 11, 20, 12, 0)
ws, s = get_measurements('DK', arrow.get(t), fetch_best_forecast(t))
print ws, s
plt.figure()
plt.hist(ws)
plt.figure()
plt.hist(s);

In [ ]:
for country_code in country_codes:
    ws_means = []
    ws_medians = []
    ws_maxs = []
    ws_mins = []
    for i in range(len(df_wind)):
        t = arrow.get(df_wind.index[i])
        w_fcast = fetch_best_forecast(t)
        if not w_fcast: 
            print 'Skipping forecast at %s' % t
            continue
        ws, _ = get_measurements(country_code, t, w_fcast)
        ws_means.append(np.mean(ws))
        ws_medians.append(np.median(ws))
        ws_maxs.append(np.max(ws))
        ws_mins.append(np.min(ws))
        
    plt.figure()
    wind_prod = df_wind.loc[:, country_code]
    plt.plot(ws_means, wind_prod, 'x')
    plt.plot(ws_medians, wind_prod, 'o')
    #plt.plot(ws_maxs, wind_prod, '+')
    #plt.plot(ws_mins, wind_prod, 'd')
    plt.title('Wind in %s' % country_code)
    plt.ylabel('Wind production (MW)')
    plt.xlabel('Wind speed (m/s)')
    plt.legend(['mean', 'median'], loc=0)

In [ ]:
for country_code in country_codes:
    s_means = []
    s_medians = []
    for i in range(len(df_solar)):
        t = arrow.get(df_solar.index[i])
        s_fcast = fetch_solar_forecast(t)
        if not s_fcast: 
            print 'Skipping forecast at %s' % t
            continue
        _, s = get_solar_measurements(country_code, t, s_fcast)
        s_means.append(np.mean(s))
        s_medians.append(np.median(s))
    
    plt.figure()
    solar_prod = df_solar.loc[:, country_code]
    plt.plot(s_means, solar_prod, 'x')
    plt.plot(s_medians, solar_prod, 'o')
    plt.title('Solar in %s' % country_code)
    plt.ylabel('Solar production (MW)')
    plt.xlabel('DSWRF (w/m^2)')
    plt.legend(['mean', 'median'], loc=0)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df_solar.loc[:,'PT'].index, s_means, 'x-')